
<a id='ifp'></a>

# The Income Fluctuation Problem with Stochastic Returns and Discounting

## Contents

- [The Income Fluctuation Problem with Stochastic Returns and Discounting](#The-Income-Fluctuation-Problem-with-Stochastic-Returns-and-Discounting)  
  - [Overview](#Overview)  
  - [The Household Problem](#The-Household-Problem)  
  - [Solution Algorithm](#Solution-Algorithm)  
  - [Implementation](#Implementation)  
  - [Exercises](#Exercises)  

In addition to what’s in Anaconda, this lecture will need the following libraries:

In [ ]:
!pip install --upgrade quantecon
!pip install interpolation

## Overview

In this lecture, we continue our study of the income fluctuation problem initiated in [an earlier lecture](ifp.ipynb).

(See that lecture for background and intuition.)

In this lecture, we introduce two significant extensions to the version we first considered:

1. Returns on assets are allowed to be stochastic.  
1. The discount factor of the agent is allowed to vary with the state.  


The first modification makes our model more realistic in terms of content and predictions.

The second modification allows us to encompass recent quantitative work using heterogeneous agent models, which often adopts this feature.

- See, for example, [[KS98]](zreferences.ipynb#krusell1998income),
  [[Cao16]](zreferences.ipynb#cao2016existence), [[HKSJ18]](zreferences.ipynb#hubmer2018comprehensive) and
  [[HN18]](zreferences.ipynb#hills2018fiscal).  


In addition to these generalizations of the income fluctuation problem, we also introduce an alternative method for solving such problems computationally.

This method is sometimes called **time iteration** and involves iterating on the Euler equation.

(In one of our lectures, we use similar ideas to solve the optimal growth model.)

This method can be shown to be globally convergent under mild assumptions, even when utility is unbounded (both above and below).

We will make use of the following imports:

In [ ]:
import numpy as np
from quantecon.optimize import brent_max, brentq
from interpolation import interp
from numba import njit
import matplotlib.pyplot as plt
%matplotlib inline
from quantecon import MarkovChain

## The Household Problem


<a id='index-1'></a>

### Set Up

A household chooses a consumption-asset path $ \{(c_t, a_t)\} $ to
maximize


<a id='equation-trans-at'></a>
$$
\mathbb E \left\{
            \sum_{t=0}^\infty
            \left(\prod_{i=0}^t \beta_i \right) u(c_t)
         \right\} \tag{1}
$$

subject to


<a id='equation-trans-at2'></a>
$$
a_{t+1} = R_{t+1} (a_t - c_t) + Y_{t+1}
\; \text{ and } \;
0 \leq c_t \leq a_t, \tag{2}
$$

with initial condition $ (a_0, Z_0)=(a,z) $ treated as given.

Note that

- $ \{R_t\}_{t \geq 1} $, the gross rate of return on wealth,  


is allowed to be stochastic.

- the constant discount factor $ \beta $ has been replaced by a discount factor process $ \{\beta_t\}_{t \geq 0} $  with $ \beta_0=1 $.  


The sequence $ \{Y_t \}_{t \geq 1} $ is non-financial income.

The stochastic components of the problem obey


<a id='equation-eq-ry-func'></a>
$$
\beta_t = \beta \left( Z_t, \epsilon_t \right),
  \quad
R_t = R \left( Z_t, \zeta_t \right),
  \quad
Y_t = Y \left( Z_t, \eta_t \right), \tag{3}
$$

where

- the maps $ \beta $, $ R $ and $ Y $ are time-invariant, nonnegative Borel-measurable functions and  
- $ \{Z_t\}_{t \geq 0} $ is an irreducible time-homogeneous Markov chain on a finite set $ \mathsf Z $  


Let $ P(z, \hat z) $ represent the probability of this exogenous state process transitioning from $ z $ to $ \hat z $ in one step.

The innovation processes $ \{\epsilon_t\} $, $ \{\zeta_t\} $ and $ \{\eta_t\} $ are IID and independent of each other.

The utility function $ u $ maps $ \mathbb R+ $ to $ \{ - \infty \} \cup \mathbb R $, is twice differentiable on $ (0, \infty) $, satisfies $ u' > 0 $ and $ u'' < 0 $ everywhere
on $ (0, \infty) $, and that $ u'(c) \to \infty $ as $ c \to 0 $ and $ u'(c) < 1 $
as $ c \to \infty $.

Regarding notation, in what follows $ \mathbb E_z \hat X $ means expectation of next period value $ \hat X $ given current value $ Z = z $

For example,

$$
\mathbb E_z \hat R
:= \mathbb E
\left[ R_{t+1} \, | \, Z_t = z \right]
$$

### Assumptions

We need restrictions to ensure that the objective [(1)](#equation-trans-at) is finite and
the solution methods described below converge.

We assume in all of what follows that the discount factor process satisfies


<a id='equation-fpbc'></a>
$$
G_\beta < 1
\quad \text{where} \quad
G_\beta := \lim_{n \to \infty}
\left(\mathbb E \prod_{t=1}^n \beta_t \right)^{1/n} \tag{4}
$$

This assumption turns out to be the most natural extension of the standard condition $ \beta < 1 $ from the constant discount case.

(You can easily confirm that $ G_\beta = \beta $ in that non-stochastic setting.)

We also need to ensure that the present discounted value of wealth
does not grow too quickly.

When $ \{R_t\} $ and $ \{ \beta_t\} $ are constant at
values $ R $ and $ \beta $, the standard restriction from the existing literature is $ \beta R < 1 $.

A natural generalization is


<a id='equation-fpbc2'></a>
$$
G_{\beta R} < 1
\quad \text{where} \quad
G_{\beta R} := \lim_{n \to \infty}
\left(\mathbb E \prod_{t=1}^n \beta_t R_t \right)^{1/n} \tag{5}
$$

Finally, we impose some routine technical restrictions on non-financial income.

$$
\mathbb E \, Y < \infty \text{ and } \mathbb E \, u'(Y) < \infty
\label{a:y0}
$$

One relatively simple setting where all these restrictions are satisfied is the CRRA environment of [[BBZ15]](zreferences.ipynb#benhabib2015).

See [[MST20]](zreferences.ipynb#ma2020income) for more details on the assumptions given above.

### Optimality

The state space for $ \{(a_t, Z_t) \}_{t \geq 0} $ is taken to be $ \mathbb S:= (0, \infty) \times \mathsf Z $.

A **feasible policy** is a Borel measurable function
$ c \colon \mathbb S \to \mathbb R $ with $ 0 \leq c(a,z) \leq a $ for all
$ (a,z) \in \mathbb S $.

A feasible policy $ c $ and initial condition
$ (a,z) \in \mathbb S $ generate an asset path
$ \{ a_t\}_{t \geq 0} $ via [(2)](#equation-trans-at2) when
$ c_t = c (a_t, Z_t) $ and $ (a_0, Z_0) = (a,z) $.

The lifetime value of policy $ c $ is


<a id='equation-vc'></a>
$$
V_c (a,z) = \mathbb E_{a,z} \sum_{t = 0}^\infty
    \beta_0 \cdots \beta_t u \left[ c (a_t, Z_t) \right] \tag{6}
$$

where $ \{ a_t\} $ is the asset path generated by $ (c,(a,z)) $.

A feasible policy $ c^* $ is called **optimal** if $ V_c \leq V_{c^*} $ on $ \mathbb S $ for any feasible policy $ c $.

A feasible policy is said to satisfy the **first order optimality  condition** if, for all $ (a,z) \in \mathbb S $,

$$
\left( u' \circ c \right)(a,z) =
\max \left\{
        \mathbb E_z \, \hat{\beta} \hat{R}
           \left( u' \circ c \right)
           \left( \hat{R} \left[ a - c(a,z)\right] + \hat{Y},
                  \, \hat{Z} \right),
        u'(a)
     \right\}
$$

This is a version of the Euler equation discussed in [add suitable reference]

(The maximization over two values is due to the possibility of corner solutions, which can occur when non-financial income is additive.)

Let $ \mathscr C $ be the space of continuous functions $ c \colon \mathbb S \to \mathbb R $ such that $ c $ is increasing in the first argument, $ 0 < c(a,z) \leq a $ for all
$ (a,z) \in \mathbb S $, and


<a id='equation-ifpc4'></a>
$$
\sup_{(a,z) \in \mathbb S}
\left| (u' \circ c)(a,z) - u'(a) \right| < \infty \tag{7}
$$

The following is proved in [[MST20]](zreferences.ipynb#ma2020income):

**Theorem.** If $ c \in \mathscr C $ and the first order optimality condition holds, then $ c $ is an optimal policy.

Now our task is to find a feasible policy satisfying the first order
optimality condition.

To do this we use time iteration, as discussed below.


<a id='ifp-computation'></a>

## Solution Algorithm


<a id='index-2'></a>

### A Time Iteration Operator

First we introduce the time iteration operator $ K $ defined
as follows:

For fixed $ c \in \mathscr C $ and $ (a,z) \in \mathbb S $, the value $ Kc(a,z) $ of the
function $ Kc $ at $ (a,z) $ is defined as the $ \xi \in (0,a] $ that solves


<a id='equation-k-opr'></a>
$$
u'(\xi) =
\max \left\{
          \mathbb E_z \, \hat{\beta} \hat{R}
             (u' \circ c)[\hat{R}(a - \xi) + \hat{Y}, \, \hat{Z}],
          \, u'(a)
       \right\} \tag{8}
$$

### Convergence Properties

When iterating in policy space, we need a way to compare two consumption policies.

To this end, we pair $ \mathscr C $ with the distance

$$
\rho(c,d)
:= \sup_{(a,z) \in \mathbb S}
          \left|
              \left(u' \circ c \right)(a,z) -
              \left(u' \circ d \right)(a,z)
          \right|,
$$

which evaluates the maximal difference in terms of marginal utility.

The benefit of this measure of distance is that, while elements of $ \mathscr C $ are not generally bounded, $ \rho $ is always finite under our assumptions.

In fact, it can be shown that

1. $ (\mathscr C, \rho) $ is a complete metric space,  
1. there exists an integer $ n $ such that $ K^n $ is a contraction
  mapping on $ (\mathscr C, \rho) $, and  
1. The unique fixed point of $ K $ in $ \mathscr C $ is
  the unique optimal policy in $ \mathscr C $.  


See [[MST20]](zreferences.ipynb#ma2020income)  for details.

We now have a clear path to successfully approximating the optimal policy:
choose some $ c \in \mathscr C $ and then iterate with $ K $ until
convergence (as measured by the distance $ \rho $)

### Testing the Assumptions

[to be added.]

## Implementation


<a id='index-3'></a>

## Exercises

Repeat earlier IFP exercises with some new twists.